In [1]:
import logging
from astrodb_utils import load_astrodb, AstroDBError
from simple.utils.spectra import check_spectrum_plottable
from astrodb_utils.fits import add_missing_keywords, add_wavelength_keywords, check_header
from astropy.io import fits
import astropy.units as u
import os
import pandas as pd
import numpy as np
from specutils import Spectrum, SpectralRegion
import specutils
from specutils.manipulation import extract_region
import os

In [10]:
path = "/Users/kelle/Hunter College Dropbox/Kelle Cruz/SIMPLE/SIMPLE-db/scripts/ingests/Zhang18/sty2054_supplemental_files"
print(os.path.exists(os.path.join(path,"SDSS_J134749.74+333601.7_sdL0_SDSS_Primeval-I.txt")))


True


In [ ]:
# Handle txt files first separately
file_plotted = 0
file_failed = 0
for filename in os.listdir(path):
    if filename.endswith(".fits") or filename.startswith("README"):
        # print(f"SKIPPING FITS file: {filename}")
        continue
    
    print(f"Reading text file: {filename}")

    file_path = os.path.join(path, filename)
    
    try:
        data = np.loadtxt(file_path, comments="#", encoding="latin1")

        # column1: #w         column2:flux          
        if (filename == "SDSS_J134749.74+333601.7_sdL0_SDSS_Primeval-I.txt"):
            wavelength = (data[:, 0] * u.AA).to(u.um)
            flux = data[:, 1] * (u.erg / (u.cm**2 * u.s * u.micron))

        # column1: #w (micron)         column2:flux          
        else:
            wavelength = (data[:, 0] * u.um)
            flux = data[:, 1] * (u.watt / u.micron/ (u.meter**2)) # Flux(lambda) in W/um/m2
        # check plottability
        spectrum = Spectrum1D(spectral_axis=wavelength, flux=flux)
        spectrum = extract_region(spectrum, SpectralRegion(0.4*u.um, 2.4*u.um))
        check_spectrum_plottable(spectrum, show_plot=True)
        file_plotted += 1
    
    except Exception as e:
        print(f"Could not read {filename}: {e}")
        file_failed += 1
print(f"Total files plotted: {file_plotted}")


In [ ]:
# calculate wavelength based on the header info
# wavelength = CRVAL1 + (i + 1 − CRPIX1) × CDELT1, 
#corrected the flux arrays matching with wavelength axis
def calculate_wavelength_micron(header, length):
    cdelt = header.get('CDELT1', header.get('CD1_1'))
    crval = header['CRVAL1']
    crpix = header['CRPIX1']

    # determine the current unit
    if crval >= 5000:
        unit = u.AA
    elif 100 <= crval < 10000:
        unit = u.nm
    else:
        unit = u.micron

    index = np.arange(length)
    wavelength = crval + (index - (crpix - 1)) * cdelt

    # convert to microns
    wavelength_micron = (wavelength * unit).to(u.micron)

    return wavelength_micron


In [ ]:
def calculate_wavelength(header):
    cdelt = header.get('CDELT1', header.get('CD1_1'))
    crval = header['CRVAL1']
    crpix = header['CRPIX1']
    naxis = header['NAXIS1']
    index = np.arange(naxis)

    wavelength = (crval + (index + 1 - crpix) * cdelt) * u.Angstrom
    return wavelength.to(u.um)

In [ ]:
# Process file with Instrument XShooter
def process_instrument_a(file_path, header, flux_data):
    flux = flux_data * u.Unit("erg / (cm2 s angstrom)")
    wavelength = calculate_wavelength(header)
    add_wavelength_keywords(header=header, wavelength_data=wavelength)
    
    spectrum = Spectrum1D(spectral_axis=wavelength, flux=flux)
    if check_spectrum_plottable(spectrum, raise_error=True, show_plot=True):
        print(f"Plotable file name (A): {os.path.basename(file_path)}")
        return True
    return False

In [25]:
# KELLES CODE
# Only do OSIRIS FITS files

file_read = 0
file_plotted = 0
file_failed = 0

for filename in os.listdir(path):
    if not filename.endswith("fits") or "OSIRIS" not in filename:
        #print(f"SKIPPING FITS file: {filename}")
        continue
    else:
        print(f"READING FITS file: {filename}")
    
    file_path = os.path.join(path, filename)
    print(f"File path: {file_path}")

    # Read the FITS file
    with fits.open(file_path, mode="update") as filehandle:
        unit_string =  "erg / (angstrom s cm2)"
        print(f"REPLACING {filehandle[0].header['BUNIT']} with {unit_string}")
        filehandle[0].header["BUNIT"] = unit_string

    try:
        spectrum = Spectrum.read(file_path, format="wcs1d-fits") #, flux_unit=u.Unit("erg / (angstrom s cm2)"))
        print(f"flux unit: {spectrum.flux.unit}")
        #if u.get_physical_type(spectrum.flux.unit) == "unknown": 
        #    spectrum.flux = spectrum.flux * u.Unit("erg / (angstrom s cm2)")  # Convert flux to erg/cm^2/s/A
        print(f"flux unit: {u.get_physical_type(spectrum.flux.unit)}, wave unit: {spectrum.spectral_axis.unit}")
        file_read +=1
    except Exception as e:
        #continue
        file_failed += 1
        raise e

    # Plot and check
    try:
        check_spectrum_plottable(spectrum, raise_error=True, show_plot=False)
        print(f"PLOTTABLE file name: {filename} \n")
        file_plotted += 1
    except Exception as e:
        continue
        file_failed += 1
        #raise e


    
    # except Exception as e:
    #     if "flux units are not expected" in str(e):
           
    #         if check_spectrum_plottable(spectrum, raise_error=True, show_plot=False):
    #             print(f"PLOTTABLE file name: {filename} \n")
    #             file_plotted += 1
    #         else:
    #             print(f"{e}")
    #             print(f"WCS SKIPPING {file_path}. \n")
    #             file_failed += 1
    #             continue
    #     else:
    #         print(f"{e}")
    #         print(f"SKIPPING {file_path}. \n")
    #         raise e
        #print(f"Error reading {filename}: {e}")
        #print(f"Header info: {header}")
        
print(f"\nTotal files read into Spectrum objects: {file_read}")
print(f"Total files plotted: {file_plotted}")
print(f"Total files failed: {file_failed}")



READING FITS file: ULAS_J231949.36+044559.5_M7_comb_OSIRIS_scombine_Primeval-IV.fits
File path: /Users/kelle/Hunter College Dropbox/Kelle Cruz/SIMPLE/SIMPLE-db/scripts/ingests/Zhang18/sty2054_supplemental_files/ULAS_J231949.36+044559.5_M7_comb_OSIRIS_scombine_Primeval-IV.fits
REPLACING erg / (angstrom s cm2) with erg / (angstrom s cm2)
flux unit: erg / (Angstrom s cm2)
flux unit: power density/spectral flux density wav, wave unit: Angstrom
PLOTTABLE file name: ULAS_J231949.36+044559.5_M7_comb_OSIRIS_scombine_Primeval-IV.fits 

READING FITS file: ULAS_J010756.85+100811.3_sdM7_OSIRIS_GTC63-13A_Primeval-IV.fits
File path: /Users/kelle/Hunter College Dropbox/Kelle Cruz/SIMPLE/SIMPLE-db/scripts/ingests/Zhang18/sty2054_supplemental_files/ULAS_J010756.85+100811.3_sdM7_OSIRIS_GTC63-13A_Primeval-IV.fits
REPLACING erg / (angstrom s cm2) with erg / (angstrom s cm2)
flux unit: erg / (Angstrom s cm2)
flux unit: power density/spectral flux density wav, wave unit: Angstrom
PLOTTABLE file name: ULAS_J

        Use textwrap.indent() instead. [astropy.io.fits.hdu.hdulist]
    Header size is not multiple of 2880: 3309
There may be extra bytes after the last HDU or the file is corrupted. [astropy.io.fits.hdu.hdulist]


flux unit: erg / (Angstrom s cm2)
flux unit: power density/spectral flux density wav, wave unit: Angstrom
PLOTTABLE file name: ULAS_J134206.86+053724.9_sdL1_OSIRIS_GTC46-14A_Primeval-IV.fits 

READING FITS file: ULAS_J135216.31+312327.0_esdL0.5_OSIRIS_GTC46-14A_Primeval-IV.fits
File path: /Users/kelle/Hunter College Dropbox/Kelle Cruz/SIMPLE/SIMPLE-db/scripts/ingests/Zhang18/sty2054_supplemental_files/ULAS_J135216.31+312327.0_esdL0.5_OSIRIS_GTC46-14A_Primeval-IV.fits
REPLACING erg / (angstrom s cm2) with erg / (angstrom s cm2)
flux unit: erg / (Angstrom s cm2)
flux unit: power density/spectral flux density wav, wave unit: Angstrom
PLOTTABLE file name: ULAS_J135216.31+312327.0_esdL0.5_OSIRIS_GTC46-14A_Primeval-IV.fits 

READING FITS file: ULAS_J143154.18-004114.3_sdM9_OSIRIS_GTC46-14A_Primeval-IV.fits
File path: /Users/kelle/Hunter College Dropbox/Kelle Cruz/SIMPLE/SIMPLE-db/scripts/ingests/Zhang18/sty2054_supplemental_files/ULAS_J143154.18-004114.3_sdM9_OSIRIS_GTC46-14A_Primeval-IV.fit

flux unit: erg / (Angstrom s cm2)
flux unit: power density/spectral flux density wav, wave unit: Angstrom
PLOTTABLE file name: ULAS_J135359.58+011856.7_sdL0_OSIRIS_GTC46-14A_Primeval-IV.fits 

READING FITS file: ULAS_J223302.03+062030.8_esdL0.5_OSIRIS_GTC39-12B_Primeval-IV.fits
File path: /Users/kelle/Hunter College Dropbox/Kelle Cruz/SIMPLE/SIMPLE-db/scripts/ingests/Zhang18/sty2054_supplemental_files/ULAS_J223302.03+062030.8_esdL0.5_OSIRIS_GTC39-12B_Primeval-IV.fits
REPLACING erg / (angstrom s cm2) with erg / (angstrom s cm2)
flux unit: erg / (Angstrom s cm2)
flux unit: power density/spectral flux density wav, wave unit: Angstrom
PLOTTABLE file name: ULAS_J223302.03+062030.8_esdL0.5_OSIRIS_GTC39-12B_Primeval-IV.fits 

READING FITS file: ULAS_J230256.53+121310.2_sdL0_OSIRIS_GTC63-13A_Primeval-IV.fits
File path: /Users/kelle/Hunter College Dropbox/Kelle Cruz/SIMPLE/SIMPLE-db/scripts/ingests/Zhang18/sty2054_supplemental_files/ULAS_J230256.53+121310.2_sdL0_OSIRIS_GTC63-13A_Primeval-IV.fit

In [ ]:
# Testing on X-Shooter instrument:
# VIS, wavelength range 559.5-1024 nm
# NIR, wavelength range 1024-2480 nm
# UVB, wavelength range 300-559.5 nm

for filename in os.listdir(path):
    if not filename.endswith("fits"):
        continue

    file_path = os.path.join(path, filename)
    try:
        with fits.open(file_path) as hdul:
            header = hdul[0].header
            flux_data = hdul[0].data

            if flux_data.shape[0] == 4 and flux_data.shape[1] == 1:
                # handle 3D flux, take the first slice 
                flux_data = flux_data[0, 0, :]

            flux = flux_data * u.Unit("erg / (cm2 s angstrom)")

            # Calculate wavelength from actual data length
            if 'CRVAL1' in header and ('CDELT1' in header or 'CD1_1' in header) and 'CRPIX1' in header:
                wavelength = calculate_wavelength_micron(header, length=len(flux_data))

            else:
                fix_header = add_missing_keywords(header)
                raise ValueError(f"Header of {filename} is missing required keywords.")

            if "xshooter" in filename.lower():
                # Add wavelength info to header (optional)
                add_wavelength_keywords(header=header, wavelength_data=wavelength)

                # Plot and check
                spectrum = Spectrum1D(spectral_axis=wavelength, flux=flux)
                if check_spectrum_plottable(spectrum, raise_error=True, show_plot=True):
                    print(f"Plotable file name: {filename}")
                    file_plotted += 1

    except Exception as e:
        print(f"Error reading {filename}: {e}")
        print(f"Header info: {header}")
        file_failed += 1
        continue

print(f"\nTotal files plotted: {file_plotted}")
print(f"Total files failed: {file_failed}")



In [ ]:
# Test for OSIRIS instrument
for filename in os.listdir(path):
    if not filename.endswith(".fits"):
        continue

    # Get fits files from sty2054 directory
    file_path = os.path.join(path, filename)

    # file info
    # fits.info(file_path)
    # print(fits.getheader(file_path))
    
    try:
        with fits.open(file_path) as hdul:
            header = hdul[0].header
            flux_data = hdul[0].data

            if flux_data is None:
                raise ValueError(f"No flux data found in {filename}")

            flux = flux_data * u.Unit("erg / (cm2 s angstrom)")
            
            instrument = header.get("INSTRUME", "").upper()
            
            if "xshooter" in filename.lower():
                print(f"Skipping Xshooter file: {filename}")
                continue
            
            # Calculate wavelength
            if 'CRVAL1' in header and 'CDELT1' in header  and 'CRPIX1' in header and 'NAXIS1' in header:
                 wavelength = calculate_wavelength(header)
            else:
                fix_header = add_missing_keywords(header)
                raise ValueError(f"Header of {filename} is missing required keywords.")
                
            # add wavelength keywords to the header if not found in file
            add_wavelength_keywords(header=header, wavelength_data=wavelength)
            
            print(f"\nReading: {filename}")
            
            # Check and plot if the new file can be read in SIMPLE
            spectrum = Spectrum1D(spectral_axis=wavelength, flux=flux)
            if check_spectrum_plottable(spectrum,raise_error=True, show_plot=True):
                print(f"Plotable file name: {filename}")
                file_plotted += 1

    except Exception as e:
        print(f"Error reading {filename}: {e}")
        print(f"Header info: {header}")
        file_failed += 1
        continue
        
print(f"\nTotal files plotted: {file_plotted}")
print(f"Total files failed: {file_failed}")